#### Import Libraries

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


#### Load and Prepare Data

In [2]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add a bias term (column of ones) to the data
X = np.c_[np.ones(X.shape[0]), X]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### Define Helper Functions

Purpose: The softmax function is used to convert raw scores (logits) into probabilities. It's often used in classification problems to represent the probability distribution over different classes.

Steps:

Stabilize Computation: Subtract the maximum value in each row of logits to prevent overflow when computing the exponentials. This is a numerical stability trick.

Exponentiate: Compute the exponentials of the stabilized logits.

Normalize: Divide by the sum of exponentials for each row to get probabilities.


Purpose: This function calculates the loss (negative log-likelihood) and gradients for the softmax regression model.

Steps:

Compute Logits: Calculate the raw scores by multiplying the feature matrix X by the parameter matrix theta.

Compute Probabilities: Apply the softmax function to the logits to get the predicted probabilities for each class.

Compute Loss:
m is the number of training examples.

np.arange(m) generates an array [0, 1, 2, ..., m-1], which represents the indices of training examples.

y_proba[np.arange(m), y] selects the predicted probabilities corresponding to the true class labels.

np.log(y_proba[np.arange(m), y]) computes the logarithm of these probabilities.

-np.mean(np.log(y_proba[np.arange(m), y])) computes the mean negative log-likelihood loss.

Compute Gradients:

np.eye(np.max(y) + 1)[y] creates a one-hot encoded matrix of the true class labels.

y_proba - np.eye(np.max(y) + 1)[y] computes the difference between the predicted probabilities and the one-hot encoded true labels.

X.T.dot(y_proba - np.eye(np.max(y) + 1)[y]) computes the gradient of the loss with respect to theta.

The gradients are averaged over all training examples by dividing by m.

Return Loss and Gradients: The function returns the computed loss and gradients

Explanation:

Purpose: This function is used to make predictions using the softmax regression model.

Steps:

Compute Logits: Calculate the raw scores by multiplying the feature matrix X by the parameter matrix theta.

Compute Probabilities: Apply the softmax function to the logits to get the predicted probabilities for each class.

Make Predictions: Use np.argmax to find the class with the highest probability for each example. This function returns the indices of the maximum values along the specified axis (in this case, axis 1, which corresponds to classes).

Summary

The softmax function converts raw scores to probabilities.

The compute_loss_and_gradients function calculates the negative log-likelihood loss and gradients for the softmax regression model.

The predict function makes predictions by finding the class with the highest predicted probability for each example.


In [3]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

def compute_loss_and_gradients(X, y, theta):
    logits = X.dot(theta)
    y_proba = softmax(logits)
    m = X.shape[0]
    entropy_loss = -np.mean(np.log(y_proba[np.arange(m), y]))
    gradients = (1/m) * X.T.dot(y_proba - np.eye(np.max(y) + 1)[y])
    return entropy_loss, gradients

def predict(X, theta):
    logits = X.dot(theta)
    return np.argmax(softmax(logits), axis=1)


#### Implement Batch Gradient Descent with Early Stopping

Function Purpose
The softmax_regression function aims to train a softmax regression model on the training data while monitoring the performance on the validation data. It uses batch gradient descent to minimize the loss function and incorporates early stopping to prevent overfitting.

Key Components

Initialization:

Input Dimensions: Determine the number of input features (n_inputs) and the number of output classes (n_outputs).

Parameter Initialization: Initialize the parameter matrix theta with random values.

Early Stopping Parameters: Initialize best_loss to infinity and epochs_without_improvement to zero. These will be used to track the best validation loss and the number of epochs without improvement, respectively.

Training Loop:

The loop runs for a maximum of n_epochs iterations.

Compute Loss and Gradients: Calculate the training loss and gradients using the compute_loss_and_gradients function.

Update Parameters: Update the parameter matrix theta using the gradient descent update rule.

Validation Loss: Calculate the loss on the validation set to monitor performance.

Early Stopping: Check if the validation loss has improved. If it has, update best_loss and reset epochs_without_improvement. If not, increment epochs_without_improvement.

Stopping Condition: If the model has not improved for patience consecutive epochs, stop training early to prevent overfitting.

Summary
Initialization: Set up dimensions, initialize parameters, and early stopping variables.

Training Loop: For each epoch:
Calculate loss and gradients on the training set.

Update parameters using gradient descent.

Calculate loss on the validation set.

Check for early stopping conditions:
If validation loss improves, update the best loss and reset the counter.
If validation loss does not improve for a specified number of epochs (patience), stop training early.

Return: The learned parameters (theta) after training completes.

In [4]:
def softmax_regression(X_train, y_train, X_val, y_val, learning_rate=0.01, n_epochs=1000, tol=1e-4, patience=5):
    n_inputs = X_train.shape[1]
    n_outputs = np.max(y_train) + 1
    theta = np.random.randn(n_inputs, n_outputs)
    
    best_loss = np.inf
    epochs_without_improvement = 0
    
    for epoch in range(n_epochs):
        loss, gradients = compute_loss_and_gradients(X_train, y_train, theta)
        theta = theta - learning_rate * gradients
        
        val_loss, _ = compute_loss_and_gradients(X_val, y_val, theta)
        
        if val_loss < best_loss - tol:
            best_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
            
    return theta

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model
theta = softmax_regression(X_train_split, y_train_split, X_val_split, y_val_split)


#### Evaluate the Model

In [5]:
# Predict and evaluate on the test set
y_pred = predict(X_test, theta)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Test accuracy: 96.67%
